# Distância e correlação

### Questionário 2 — Processos Dinâmicos em Redes Complexas

Giovana Daniele da Silva, 10692224

____

In [1]:
import math
import igraph
import numpy as np
from scipy.stats import pearsonr
from collections import defaultdict

In [2]:
def get_net(path, directed=False, weighted=False):
    g = igraph.Graph.Read_Ncol(path, directed=directed, weights=weighted)
    g = g.simplify()
    
    cl = g.clusters()
    lcc = cl.giant()
    
    return lcc

___

#### 1. Para a rede social Hamsterster, calcule a média dos menores caminhos e o diâmetro. Considere apenas o maior componente.

In [3]:
g = get_net("data/hamsterster.txt")

In [4]:
print("Média dos menores caminhos:", round(np.mean(g.shortest_paths()), 1))
print("Diâmetro:", g.diameter())

Média dos menores caminhos: 3.5
Diâmetro: 14


___

#### 2. Para a rede de aeroportos (base USairport500), calcule a média e variância do comprimento dos menores caminhos. Considere apenas o maior componente na rede.

In [5]:
g = get_net("data/USairport500.txt")

In [6]:
print("Média do comprimento dos menores caminhos:", round(np.mean(g.shortest_paths()), 2))
print("Variância do comprimento dos menores caminhos:", round(np.var(g.shortest_paths()), 2))

Média do comprimento dos menores caminhos: 2.99
Variância do comprimento dos menores caminhos: 0.83


____

#### 3. Calcule o coeficiente de assortatividade para a rede Advogato. Considere apenas o maior componente.

In [7]:
g = get_net("data/advogato.txt")

In [8]:
print("Assortatividade:", round(g.assortativity_degree(directed=False), 3))

Assortatividade: -0.096


___

#### 4. Para a rede de aeroportos (USairport500), calcule a entropia de Shannon considerando o comprimento dos menores caminhos. Use o logaritmo na base 2 e considere apenas o maior componente.

In [9]:
g = get_net("data/USairport500.txt")

In [10]:
def shortest_path_distribution(G):
    vk = np.array([shortest_path for node in g.shortest_paths() for shortest_path in node])
    
    maxk = np.max(vk)
    mink = np.min(vk)
    
    kvalues= np.arange(0, maxk+1)
    Pk = np.zeros(maxk+1)
    
    for k in vk:
        Pk[k] = Pk[k] + 1
        
    Pk = Pk/sum(Pk)
    
    return kvalues, Pk

In [11]:
def shannon_entropy(G):
    k, Pk = shortest_path_distribution(G)
    H = 0
    
    for p in Pk:
        if(p > 0):
            H = H - p*math.log(p, 2)
            
    return H

In [12]:
print("Entropia de Shannon considerando o comprimento dos menores caminhos:",
      round(shannon_entropy(g.shortest_paths()), 2))

Entropia de Shannon considerando o comprimento dos menores caminhos: 1.9


___

#### 5. Calcule o coeficiente de correlação de Pearson entre o grau médio dos vizinhos e o grau para a rede de palavras (word_adjacencies). Considere apenas o maior componente conectado.

In [13]:
g = get_net("data/word_adjacencies.txt")

In [14]:
def get_degree_and_avg_degree_of_neighbors_of_degree(G):
    degree_of_neighbors_of_degree = defaultdict(list)
    avg_degree_neighbors = defaultdict(int)

    for node in G.vs:
        degree_of_neighbors_of_degree[node.degree()].extend(G.degree(G.neighbors(node)))

    for degree in degree_of_neighbors_of_degree.keys():
        avg_degree_neighbors[degree] = np.mean(degree_of_neighbors_of_degree[degree])
        
    return list(avg_degree_neighbors.keys()), list(avg_degree_neighbors.values())

In [15]:
degrees, avg_degree_of_neighs = get_degree_and_avg_degree_of_neighbors_of_degree(g)

In [16]:
print("A correlação entre o grau e o grau médio dos vizinhos do grau é:", 
      round(pearsonr(degrees, avg_degree_of_neighs)[0], 2))

A correlação entre o grau e o grau médio dos vizinhos do grau é: -0.71
